In [ ]:
!pip install -q --upgrade transformers torch torchvision torchaudio
!pip install -q tokenizers==0.13.3 evaluate
!pip install -q bitsandbytes transformers accelerate gradio thread6

- similar to pandas, 🤗 Datasets, provides several functions to manipulate the contents of 'Datset', and 'DatasetDict' objects
- We already encountered the 'Dataset.map()' method in chapter 3
- For this example we use the 'Drug Review Datset' from UC Irvine ML Repo which contains patient reviews on various drugs

In [4]:
# create data directory
!mkdir data

# we gon be downloading shit so lets change our directory to the data folder we created
%cd data

# we download the data using the wget command
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"

# its in .zip format so we'll unzip it as well
!unzip drugsCom_raw.zip

/notebooks/NLP_huggingface/Chapter_5/data
--2023-09-18 22:06:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [               <=>  ]  41.00M   685KB/s    in 24s     

2023-09-18 22:07:15 (1.68 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [5]:
# we got .tsv files! 
# wtf are those?
# From what I read, they're a variant of csv files
# the differnece is they use a tab instead of a comma so hence the 't' instead of a 'c' in 'tsv'

# to use it with the 'load_dataset()' method, we just have to specify the delimiter with the, you guessed it, 'delimiter' argument
from datasets import load_dataset

data_files = {'train': 'drugsComTrain_raw.tsv', 'test': 'drugsComTest_raw.tsv'}

# '\t' is the tab character in python
drug_dataset = load_dataset('csv', data_files = data_files, delimiter = '\t')# notice the delimiter

Using custom data configuration default-f13cdf9f91e6421f


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

/usr/local/lib/python3.9/dist-packages/datasets/download/streaming_download_manager.py:695: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


0 tables [00:00, ? tables/s]

/usr/local/lib/python3.9/dist-packages/datasets/download/streaming_download_manager.py:695: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f13cdf9f91e6421f/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# good practice is to get a random sample from your dataset to make sure its all good

# Theres a cool trick where you can get a random sample by chaining 'Dataset.shuffle()' and 'Dataset.select()'
drug_sample = drug_dataset['train'].shuffle(seed = 42).select(range(1000))

# peek at first few 
print(drug_sample[:3])

# column names of all drug_samples
drug_sample.column_names

{'Unnamed: 0': [87571, 178045, 80482], 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'], 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'], 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"', '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."', '"I have been taking Mobic for over a year with no side effects other than an elev

['Unnamed: 0',
 'drugName',
 'condition',
 'review',
 'rating',
 'date',
 'usefulCount']

- Note the above has been fixed purposely for reproducibility,
- "Dataset.select()" expects an iterable of indices, so we've passed range(1000) to grab the ifrst 1,000 examples from the shuffled dataset.
- The sample let's us see a few quirks in our dataset:
    - The Unnamed: 0 column looks suspiciously like an ananymized ID for each patient
    - The condition column includes a mix of uppercase and lowercase labels
    - The reviews are of varying length and contain a mix of Python line seperators (\r\n) as well as HTML character code like (&\#039)

In [8]:
# We can use the 'Dataset.unique()' function to verify that the number of IDs matches the number of rows in each split
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

t
